In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
sns.set(font_scale=1)

import warnings
warnings.filterwarnings('ignore')

In [4]:
def to_tokenize(text_Series, dict_change_word): # input은 '기사내용'을 pd.Series 타입으로 줌. out도 Series 타입.
    import re

    # 1. URL 부분 제거.
    print("\nURL 제거 과정.\n")
    text_Series = [re.sub('((http(s?))\:\/\/)([0-9a-zA-Z\-]+\.*\/*(\\n)*)+|url|URL', '', text) for text in tqdm(text_Series)]

    # 2. e-mail 주소 제거.
    print("\ne-mail 제거 과정.\n")
    text_Series = [re.sub('(\<.[a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4})', '', text) for text in tqdm(text_Series)] 

    # 3. 고유어 변경
    print("\n고유어 변경 과정.\n")
    #text_Series = text_Series.apply(lambda x: change_hiphen_word(x, dict_change_word))
    text_Series = [change_hiphen_word(text, dict_change_word) for text in tqdm(text_Series)]
    
    # 4. 특수문자 제거. 모두 제거.
    print("\n특수문자 제거 과정.\n")
    text_Series = [re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', text) for text in tqdm(text_Series)]

    # 5. 문자 외 모두 제거(공백, \, 숫자 사라짐)
    print("\n문자 외 제거 과정.\n")
    text_Series = [re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', text) for text in tqdm(text_Series)]

    return text_Series


In [5]:
#dict_change_word에 해당하는 단어를 변경하는 함수

def change_hiphen_word(text, dictionary):
    com_list = [".replace('{}', '{}')".format(word, dictionary[word]) for word in dictionary.keys() if word in text]
    com_list.insert(0, "'{}'.format(text)")
    command = ''.join(com_list)
    text = eval(command)

    return text

In [6]:
#Corpus 만드는 함수 (딕셔너리 반영)
def make_corpus(text_col):
    from gensim import corpora
    
    dictionary = pd.read_pickle('dictionary.pkl')
    
    corpus = [dictionary.doc2bow(text) for text in text_col]
    
    return dictionary, corpus

In [110]:
#하나로 합쳐진 데이터 불러오기

news_entire = pd.read_csv('news_clean.csv', encoding = 'cp949')

news_company = []

news_list = ['CSR','과징금','공정위','지역사회','지원','중소기업','상생','협력사','갑질',
             '봉사','기부','ESG','사망','사고','취업','과태료','유출','취약계층','산재']

for i in news_list:
    news_company.append(news_entire[news_entire.title.str.contains(i) == True])

news_company = pd.concat(news_company)    

news_company = news_company.drop_duplicates(['contents','company'])

news_company

,Unnamed: 0,date,title,contents,company
791,76833,2020.12.09.,"CSR포럼·국토부, CSR 언택트 정기포럼",또 △김주헌 GGGI 필리핀 사무소장 △나영훈 포스코건설 기업시민사무국 리더 △윤한...,CJ대한통운
983,77139,2020.08.17.,"'책 읽는 외국어 마을'…CJ대한통운, 글로벌 임직원과 '뉴노멀 CSR' 진행",뉴노멀 시대를 맞아 CJ대한통운의 사회공헌활동이 변화하고 있다. CJ대한통운은 '뉴...,CJ대한통운
1989,77568,2020.10.26.,"[2020 CSR]CJ, 지역 식품업체 성장 돕는 ‘즐거운 동행’ 실천",다양한 사회공헌 활동을 이어가고 있다. CJ제일제당은 ‘사업보국’ 이념과 ‘지속 가...,CJ제일제당
2041,77656,2020.08.31.,"[하반기 기업 CSR 동향, 소비재 편 ④] LG생활건강, SPC, 이해관계자 중심...",소비재 기업들은 판매 제품이 가장 필요한 이해관계자에 전달되도록 사회공헌활동을 설계...,CJ제일제당
2806,143,2020.12.23.,"[뉴스락 2020 연중기획┃함께 가자 우리 이길을 ② 보험 CSR 편] ""앞으로도 ...",CSR(Corporate Social Respensibility)이란 기업의 사회적...,DB손해보험
...,...,...,...,...,...
56273,2,2020.05.28.,산재예방 낙제생에서 모범생으로…한전·LH·농어촌공사 비결은?,"한국가스공사와 한국동서발전은 산업재해에 노출되기 쉬운 작업 환경에도 불구, 철저한 ...",한국가스공사
56299,0,2020.12.27.,"이산 산재보상센터, ""거제 지역 근로자 근골격계 질환 산재승인 위해 노력""",거제 지역에는 대우조선해양 및 삼성중공업 등 대기업의 조선소가 입주하여 조선업이 활...,대우조선해양
56304,0,2020.12.27.,"이산 산재보상센터, ""거제 지역 근로자 근골격계 질환 산재승인 위해 노력""",거제 지역에는 대우조선해양 및 삼성중공업 등 대기업의 조선소가 입주하여 조선업이 활...,삼성중공업
56305,6,2020.09.14.,"""산재를 겪으면서 세상이 내 편이 아니라 느꼈습니다""",삼성중공업 크레인 사고 산재 트라우마 피해자 김영환 노동 현장에서 업무상 사고나 질...,삼성중공업


In [109]:
#딕셔너리 통합 (고유어들을 하나의 주제로 분류할 수 있게끔 설정)

dict_change_word= {'경북':'지역사회','광주':'지역사회','대구' : '지역사회' 
                   ,'외국인':'취약계층', '장애인':'취약계층',
                   '사회적 책임':'사회공헌','CSR':'사회공헌', '고객정보':'개인정보',
                   'ESG':'지속가능', '이노베이션':'혁신',
                   '동반성장' : '상생', '갑질' : '불공정', 
                   '산재':'산업재해', '산업 재해' : '산업재해'}

In [71]:
#토큰화하기 위해 데이터클리닝을 해줌

news_entire_to_tokenize = pd.DataFrame(to_tokenize(news_company['title'], dict_change_word))

news_entire_to_tokenize.reset_index(drop=True, inplace=True)

news_entire_to_tokenize.head(10)

 31%|██████████████████████▏                                                 | 10839/35133 [00:00<00:00, 107489.04it/s]


URL 제거 과정.


e-mail 제거 과정.


고유어 변경 과정.



 79%|█████████████████████████████████████████████████████████▏              | 27908/35133 [00:00<00:00, 278054.71it/s]


특수문자 제거 과정.


문자 외 제거 과정.



100%|████████████████████████████████████████████████████████████████████████| 35133/35133 [00:00<00:00, 268499.59it/s]


,0
0,사회공헌포럼 국토부 사회공헌 언택트 정기포럼
1,책 읽는 외국어 마을 대한통운 글로벌 임직원과 뉴노멀 사회공헌 진행
2,사회공헌 지역 식품업체 성장 돕는 즐거운 동행 실천
3,하반기 기업 사회공헌 동향 소비재 편 생활건강 이해관계자 중...
4,뉴스락 연중기획 함께 가자 우리 이길을 보험 사회공헌 편 앞으로도...
5,뉴스락 연중기획 함께 가자 우리 이길을 보험 사회공헌 편 앞으로도...
6,사회공헌 고민 노력 김남호 회장 취임 한달만에 성금 억 지원
7,투자로 말하는 사회공헌 대한항공 발 빠른 지속가능경영에 사회...
8,투자로 말하는 사회공헌 대한항공 발 빠른 지속가능경영에 사회...
9,대그룹 사회공헌 결산 그룹 환경 에 방점 건설 칼텍스는...


In [89]:
#Stopword에 추가할 리스트 (조사, 명사, 일부 동사)

stopword =  pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt")['아'].tolist()

stopwords_post = ['이', '가', '께서', '에서', '서', '이다', '의', '을', '를', '이', '가', '에', '에게', '께', '한테', '에서', '에게서', '로서', '로써', '고', '이라고', '라고', '와', '과', '이랑', '에', '같이', '처럼', '만큼', '만치', '야',
                  '아', '여', '이여', '이시여', '와', '과', '하고', '이다', '이며', '에다', '에다가', '이랑', '랑']

my_stopwords = pd.read_table("stopword_list.txt")['목록'].values.tolist()

stopword = set(stopword + stopwords_post + my_stopwords)

In [73]:
#Tagging 작업

from konlpy.tag import Okt

okt = Okt()

news_tagged = news_entire_to_tokenize[0].apply(lambda x : okt.morphs(x, stem = True)).tolist()

#불용어 제거

news_tagged = [[word for word in word_list if word not in stopword] for word_list in tqdm(news_tagged)]

#한글자 제거

news_tagged = [[word for word in word_list if len(word) > 1] for word_list in tqdm(news_tagged)]

100%|████████████████████████████████████████████████████████████████████████| 35133/35133 [00:00<00:00, 611334.37it/s]


In [90]:
#태깅 완료 후 제목을 태깅한 것으로 대체해줌

news_ready_for_LDA = news_company
news_ready_for_LDA['title'] = news_tagged

news_ready_for_LDA

,Unnamed: 0,date,title,contents,company
791,76833,2020.12.09.,"[사회, 공헌, 포럼, 국토부, 사회, 공헌, 언택트, 정기, 포럼]",또 △김주헌 GGGI 필리핀 사무소장 △나영훈 포스코건설 기업시민사무국 리더 △윤한...,CJ대한통운
983,77139,2020.08.17.,"[읽다, 외국어, 마을, 글로벌, 직원, 노멀, 사회, 공헌, 진행]",뉴노멀 시대를 맞아 CJ대한통운의 사회공헌활동이 변화하고 있다. CJ대한통운은 '뉴...,CJ대한통운
1989,77568,2020.10.26.,"[사회, 공헌, 지역, 식품, 업체, 성장, 돕다, 즐겁다, 동행, 실천]",다양한 사회공헌 활동을 이어가고 있다. CJ제일제당은 ‘사업보국’ 이념과 ‘지속 가...,CJ제일제당
2041,77656,2020.08.31.,"[하반기, 기업, 사회, 공헌, 동향, 소비재, 생활, 건강, 이해관계자, 중심]",소비재 기업들은 판매 제품이 가장 필요한 이해관계자에 전달되도록 사회공헌활동을 설계...,CJ제일제당
2806,143,2020.12.23.,"[연중, 기획, 가다, 이기다, 사회, 공헌, 으로도]",CSR(Corporate Social Respensibility)이란 기업의 사회적...,DB손해보험
...,...,...,...,...,...
56273,2,2020.05.28.,"[산업, 재해, 예방, 낙제, 모범생, 농어촌, 공사, 비결]","한국가스공사와 한국동서발전은 산업재해에 노출되기 쉬운 작업 환경에도 불구, 철저한 ...",한국가스공사
56299,0,2020.12.27.,"[이산, 산업, 재해, 보상, 센터, 거제, 지역, 근로자, 골격계, 질환, 산업,...",거제 지역에는 대우조선해양 및 삼성중공업 등 대기업의 조선소가 입주하여 조선업이 활...,대우조선해양
56304,0,2020.12.27.,"[이산, 산업, 재해, 보상, 센터, 거제, 지역, 근로자, 골격계, 질환, 산업,...",거제 지역에는 대우조선해양 및 삼성중공업 등 대기업의 조선소가 입주하여 조선업이 활...,삼성중공업
56305,6,2020.09.14.,"[산업, 재해, 겪다, 세상, 편이, 아니다, 느끼다]",삼성중공업 크레인 사고 산재 트라우마 피해자 김영환 노동 현장에서 업무상 사고나 질...,삼성중공업


In [91]:
#전체 데이터프레임 ['기사내용']에 전처리 완료된 데이터 저장
news_ready_for_LDA.to_pickle('news_ready_for_LDA.pkl')

In [92]:
#데이터 클리닝

news_ready_for_LDA = pd.read_pickle('news_ready_for_LDA.pkl')

news_ready_for_LDA.drop(news_ready_for_LDA.loc[news_ready_for_LDA['date']=='CHIEF EXECUTIVE'].index, inplace=True)
news_ready_for_LDA.drop(news_ready_for_LDA.loc[news_ready_for_LDA['date']=='date'].index, inplace=True)
news_ready_for_LDA['date'] = pd.to_datetime(news_ready_for_LDA['date'], format = "%Y.%m.%d.")

In [93]:
news_ready_for_LDA.reset_index(drop=True, inplace=True)

news_ready_for_LDA

,Unnamed: 0,date,title,contents,company
0,76833,2020-12-09,"[사회, 공헌, 포럼, 국토부, 사회, 공헌, 언택트, 정기, 포럼]",또 △김주헌 GGGI 필리핀 사무소장 △나영훈 포스코건설 기업시민사무국 리더 △윤한...,CJ대한통운
1,77139,2020-08-17,"[읽다, 외국어, 마을, 글로벌, 직원, 노멀, 사회, 공헌, 진행]",뉴노멀 시대를 맞아 CJ대한통운의 사회공헌활동이 변화하고 있다. CJ대한통운은 '뉴...,CJ대한통운
2,77568,2020-10-26,"[사회, 공헌, 지역, 식품, 업체, 성장, 돕다, 즐겁다, 동행, 실천]",다양한 사회공헌 활동을 이어가고 있다. CJ제일제당은 ‘사업보국’ 이념과 ‘지속 가...,CJ제일제당
3,77656,2020-08-31,"[하반기, 기업, 사회, 공헌, 동향, 소비재, 생활, 건강, 이해관계자, 중심]",소비재 기업들은 판매 제품이 가장 필요한 이해관계자에 전달되도록 사회공헌활동을 설계...,CJ제일제당
4,143,2020-12-23,"[연중, 기획, 가다, 이기다, 사회, 공헌, 으로도]",CSR(Corporate Social Respensibility)이란 기업의 사회적...,DB손해보험
...,...,...,...,...,...
35128,2,2020-05-28,"[산업, 재해, 예방, 낙제, 모범생, 농어촌, 공사, 비결]","한국가스공사와 한국동서발전은 산업재해에 노출되기 쉬운 작업 환경에도 불구, 철저한 ...",한국가스공사
35129,0,2020-12-27,"[이산, 산업, 재해, 보상, 센터, 거제, 지역, 근로자, 골격계, 질환, 산업,...",거제 지역에는 대우조선해양 및 삼성중공업 등 대기업의 조선소가 입주하여 조선업이 활...,대우조선해양
35130,0,2020-12-27,"[이산, 산업, 재해, 보상, 센터, 거제, 지역, 근로자, 골격계, 질환, 산업,...",거제 지역에는 대우조선해양 및 삼성중공업 등 대기업의 조선소가 입주하여 조선업이 활...,삼성중공업
35131,6,2020-09-14,"[산업, 재해, 겪다, 세상, 편이, 아니다, 느끼다]",삼성중공업 크레인 사고 산재 트라우마 피해자 김영환 노동 현장에서 업무상 사고나 질...,삼성중공업


In [94]:
from gensim import corpora, models

#ldamodel = models.wrappers.LdaMallet.load(datapath("model"))

ldamodel = pd.read_pickle('lda_model.pkl')

dictionary, corpus = make_corpus(news_ready_for_LDA['title'])

len(corpus)

35133

In [95]:
#토픽을 각 열로 옮김

topic_list = []

for i, j in enumerate(ldamodel[corpus]):
    prob_list = []
    temp = j
    
    for k, (topic_num, prop_topic) in enumerate(temp):
        prob_list.append(prop_topic)
    
    
    prob_df = pd.DataFrame(prob_list)
    
    prob_df_after_choice = [prob_df == max(prob_list)][0].T.astype(int)
    
    prob_df_after_choice = prob_df_after_choice / sum(prob_df_after_choice.T[0])
    
    topic_list.append(prob_df_after_choice)
    

In [96]:
topic_df = pd.concat(topic_list)
topic_df.reset_index(drop=True, inplace=True)

In [97]:
news_topic_df = pd.concat([news_ready_for_LDA['date'], topic_df, news_ready_for_LDA['company']],axis=1)

In [98]:
#각 날짜에 해당하는 모든 기사 Topic 정보 합치는 함수
def grouped_by_date(concat_df, num_topics):
    grouped_df = concat_df[[x for x in range(num_topics)]].groupby(concat_df['year'])
    
    grouped_with_sum = grouped_df.sum()
    
    grouped_with_sum['합계'] = grouped_with_sum.sum(axis=1)
    
    return grouped_with_sum

In [99]:
def normalized_topic(topic_sum_df):
    topic_sum_df2 = topic_sum_df.iloc[0:,0:9]
    for i in range(len(topic_sum_df2)):
        topic_sum_df2.iloc[i] = topic_sum_df2.iloc[i] / topic_sum_df2.iloc[i, -1]
        
    topic_sum_df2['company'] = topic_sum_df['company']
    return topic_sum_df2

In [100]:
company_df = []
company_list = []

company = pd.read_table("company_list.txt")['기업명'].values.tolist()

news_topic_df = news_topic_df.dropna()

for i in company:
    
    company_input = news_topic_df[news_topic_df['company'] == i]
    
    if len(company_input) != 0:
        company_df.append(news_topic_df[news_topic_df['company'] == i])
        company_list.append(i)


In [101]:
company_df

[            date    0    1     2     3    4    5     6     7 company
 274   2020-02-17  0.0  0.0  0.00  0.00  0.0  0.0  1.00  0.00  BGF리테일
 275   2020-02-14  0.0  0.0  0.00  0.00  0.0  0.0  1.00  0.00  BGF리테일
 276   2020-02-14  0.0  0.0  0.00  0.00  0.0  0.0  1.00  0.00  BGF리테일
 277   2020-02-14  0.0  0.0  0.00  0.00  1.0  0.0  0.00  0.00  BGF리테일
 278   2020-02-14  0.0  0.0  0.00  0.00  0.0  0.0  1.00  0.00  BGF리테일
 ...          ...  ...  ...   ...   ...  ...  ...   ...   ...     ...
 29169 2020-07-24  0.0  0.0  0.25  0.25  0.0  0.0  0.25  0.25  BGF리테일
 32311 2020-04-10  0.0  0.0  0.00  1.00  0.0  0.0  0.00  0.00  BGF리테일
 34121 2020-12-23  0.0  1.0  0.00  0.00  0.0  0.0  0.00  0.00  BGF리테일
 34511 2020-07-06  1.0  0.0  0.00  0.00  0.0  0.0  0.00  0.00  BGF리테일
 34512 2020-06-17  0.0  0.0  0.00  1.00  0.0  0.0  0.00  0.00  BGF리테일
 
 [396 rows x 10 columns],
             date    0         1    2    3    4         5    6         7  \
 2543  2020-05-28  0.0  0.000000  0.0  0.0  0.0  0.00000

In [102]:
news_topic_normalized=[]

for sub in company_df:
    sub['year'] = sub['date'].dt.year
    ##def 실행 - 날짜 기준으로 기사의 topic 정보 합침
    
    temp = grouped_by_date(sub, 8)
    temp['company'] = sub['company'].iloc[0]
    
    news_topic_normalized.append(normalized_topic(temp))

In [103]:
news_topic_normalized = pd.concat(news_topic_normalized)

news_topic_normalized.reset_index(drop=False, inplace=True)

In [104]:
# topic 컬럼 정리

columns = ["date"] + ["Topic_{}".format(i) for i in range(8)] + ['합계'] + ['company']
news_topic_normalized.columns = columns

In [105]:
news_topic_normalized

,date,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,합계,company
0,2020,0.039983,0.280724,0.103535,0.048822,0.042298,0.126684,0.236742,0.121212,1.0,BGF리테일
1,2020,0.236842,0.067251,0.210526,0.315789,0.000000,0.102339,0.035088,0.032164,1.0,BNK금융지주
2,2020,0.000000,0.000000,0.625000,0.000000,0.000000,0.250000,0.125000,0.000000,1.0,BYC
3,2020,0.125000,0.239583,0.218750,0.177083,0.000000,0.114583,0.062500,0.062500,1.0,CJ CGV
4,2020,0.100976,0.111486,0.071321,0.239865,0.300113,0.067755,0.019332,0.089152,1.0,CJ대한통운
...,...,...,...,...,...,...,...,...,...,...,...
255,2020,0.444444,0.203704,0.000000,0.166667,0.000000,0.148148,0.037037,0.000000,1.0,효성화학
256,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,휠라홀딩스
257,2020,0.058824,0.147059,0.117647,0.176471,0.058824,0.382353,0.000000,0.058824,1.0,휴비스
258,2020,0.000000,0.500000,0.000000,0.166667,0.000000,0.000000,0.333333,0.000000,1.0,휴켐스


In [106]:
news_topic_normalized.to_csv('topic_variable.csv', encoding = 'UTF-8')

In [107]:
news_topic_normalized.to_excel('topic_variable.xlsx')